In [6]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import pathlib
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from models import DataTrustedIdentifier, InfoImage
from database import connect
from sqlalchemy import and_
from sqlalchemy.orm import Session

def text_bold(string):
    return '\033[1m' + string + '\033[0m'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Database connection

In [2]:
session: Session
engine, session = connect()
engine.echo=False

2023-03-19 18:49:27,132 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 18:49:27,132 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 18:49:27,134 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 18:49:27,134 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 18:49:27,135 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 18:49:27,135 INFO sqlalchemy.engine.Engine [raw sql] {}


In [3]:
list_images = [image for image in pathlib.Path('/home/xandao/Documentos/dataset_gimp/imagens_br/imagens/RGB/specific_epithet/256/5').rglob('*.jpeg') if image.is_file()]
len(list_images)

10516

In [4]:
list_species = []
list_seq = []
list_path_image = []

for i, image in enumerate(list_images):
    print('image %d of %d' % (i, len(list_images)))
    q = session.query(DataTrustedIdentifier.specific_epithet, DataTrustedIdentifier.seq, InfoImage.path_image)\
        .filter(and_(InfoImage.path_image.ilike(sa.func.concat('%',str(image.name),'%')),
                     DataTrustedIdentifier.seq == InfoImage.seq_id))\
        .distinct(DataTrustedIdentifier.specific_epithet)\
        .all()
    if len(q) > 1:
        print(image.name, q)
        break
    for result in q:
        species = result[0]
        seq = result[1]
        path_image = result[2]
        list_species.append(species)
        list_seq.append(seq)
        list_path_image.append(path_image)

image 0 of 10516
image 1 of 10516
image 2 of 10516
image 3 of 10516
image 4 of 10516
image 5 of 10516
image 6 of 10516
image 7 of 10516
image 8 of 10516
image 9 of 10516
image 10 of 10516
image 11 of 10516
image 12 of 10516
image 13 of 10516
image 14 of 10516
image 15 of 10516
image 16 of 10516
image 17 of 10516
image 18 of 10516
image 19 of 10516
image 20 of 10516
image 21 of 10516
image 22 of 10516
image 23 of 10516
image 24 of 10516
image 25 of 10516
image 26 of 10516
image 27 of 10516
image 28 of 10516
image 29 of 10516
image 30 of 10516
image 31 of 10516
image 32 of 10516
image 33 of 10516
image 34 of 10516
image 35 of 10516
image 36 of 10516
image 37 of 10516
image 38 of 10516
image 39 of 10516
image 40 of 10516
image 41 of 10516
image 42 of 10516
image 43 of 10516
image 44 of 10516
image 45 of 10516
image 46 of 10516
image 47 of 10516
image 48 of 10516
image 49 of 10516
image 50 of 10516
image 51 of 10516
image 52 of 10516
image 53 of 10516
image 54 of 10516
image 55 of 10516
im

In [9]:
pd.DataFrame({
    'species': list_species,
    'seq': list_seq,
    'path': list_path_image
})

,species,seq,path
0,fonteboanum,32655,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
1,fonteboanum,36530,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
2,fonteboanum,37064,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
3,fonteboanum,32649,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
4,fonteboanum,39915,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
...,...,...,...
10511,aequale,30447,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
10512,aequale,37264,/home/xandao/Documentos/RGB/512/w_pred_mask/NY...
10513,aequale,45718,/home/xandao/Documentos/RGB/512/w_pred_mask/IN...
10514,aequale,9316,/home/xandao/Documentos/RGB/512/w_pred_mask/MB...


In [ ]:
session.close()
engine.close()